# Python批量处理文件

## 1、随机生成电话号码

In [ ]:
# 确定电话号段
"""
中国电信号段：133,153,180,181,189,173,177,149
中国联通号段：130,131,132,155,156,185,186，145,176
中国移动号码：134,135,136,137,138,139,150,151,152,158,159,182,183,184,147,178
"""

In [ ]:
#发现总结电话号码的规律
"""
电话号码共计11位
第一位：1
第二位：3,4,5,7,8
第三位：根据第二位来确定
    3 + 【0-9】
    4 + 【5,7,9】
    5 + 【0-9】！4
    7 + 【1-9】！4and9
    8 + 【0-9】
后8位：随机生成8个数字
"""

In [15]:
import random

#生成电话号码
def creat_phone_num():
    #第二位
    second = [3,4,5,7,8][random.randint(0,4)]
    
    #第三位：根据第二位来确定
    third = {
        3:random.randint(0,9),
        4:[5,7,9][random.randint(0,2)],
        5:[i for i in range(10) if i != 4][random.randint(0,8)],
        7:[i for i in range(10) if i not in [4,9]][random.randint(0,7)],
        8:random.randint(0,9)
    }[second]
    
    #后8位：随机生成8个数字
    suffix = ""
    for i in range(8):
        suffix = suffix + str(random.randint(0,9))
    
    #拼接
    return "1{}{}{}".format(second,third,suffix)

#调用

num = input("请输入生成的数量")

for index in range(0,int(num)):
    print(creat_phone_num())

请输入生成的数量3
14594208992
13935936579
17816873255


## 2、拆分csv文件
'''
读取当前csv 
确定一下分给多少人 ， 每个是多少   100
获取数值 
'''

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('./ratings.csv')

person = ['zhanbsan','lisi','wangwu','zhaoliu','lila']

size = data.shape[0] / len(person)

if data.shape[0] % len(person) != 0:
    size = int(np.ceil(size))
else:
    size = int(size)
    

for idx,person_name in enumerate(person):
    begin = idx * size
    end = begin + size
    df = data.iloc[begin:end]
    
    filepath = './output/ratings_{}.csv'.format(person_name)
    
    df.to_csv(filepath,index=False)

## 3、合并csv文件

In [18]:
df1 = pd.read_csv('./output/ratings/rating_zs.csv')
df1.set_index('Unnamed: 0',inplace=True)
df1.index.name = None
df1

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
25204,177,2719,3.0
25205,177,2720,2.0
25206,177,2722,2.0
25207,177,2724,3.5


In [19]:
df2 = pd.read_csv('./output/ratings/rating_lisi.csv')
df2.set_index('Unnamed: 0',inplace=True)
df2.index.name = None
df2

,userId,movieId,rating
25209,177,2729,3.0
25210,177,2746,4.0
25211,177,2761,2.0
25212,177,2791,3.0
25213,177,2797,5.0
...,...,...,...
50413,325,3927,3.0
50414,325,3981,2.0
50415,325,3994,4.0
50416,325,4017,3.0


In [20]:
df3 = pd.read_csv('./output/ratings/rating_w5.csv')
df3.set_index('Unnamed: 0',inplace=True)
df3.index.name = None
df3

,userId,movieId,rating
50418,325,4085,3.0
50419,325,4086,3.0
50420,325,4117,4.0
50421,325,4226,4.0
50422,325,4239,4.0
...,...,...,...
75622,477,2046,4.0
75623,477,2053,3.0
75624,477,2054,4.0
75625,477,2064,4.0


In [21]:
df4 = pd.read_csv('./output/ratings/rating_zl.csv')
df4.set_index('Unnamed: 0',inplace=True)
df4.index.name = None
df4

,userId,movieId,rating
75627,477,2072,3.5
75628,477,2081,3.0
75629,477,2082,3.5
75630,477,2085,4.0
75631,477,2093,4.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


###  方法一：封装一个函数，读取文件数据

In [ ]:
def read_Data(filePath,size=10000):
    
    # 使用迭代器的方式读取数据
    df = pd.read_csv(filePath,header=0,iterator=True)
    
    # 保存数据块的列表
    chunks = []
    
    # 循环起始值
    loop = True
    
    while loop:
        try:
            chunk = df.get_chunk(size)   
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
            
    df = pd.concat(chunks,ignore_index=True)
    return df

df = []
df.append(read_Data(filePath='./output/ratings/rating_zs.csv'))
df.append(read_Data(filePath='./output/ratings/rating_lisi.csv'))
df.append(read_Data(filePath='./output/ratings/rating_w5.csv'))
df.append(read_Data(filePath='./output/ratings/rating_zl.csv'))

df = pd.concat(df,ignore_index=True)
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.to_csv('./output/ratings/ratings.csv')
df

###  方法二：封装一个函数，批量处理（更佳）
思路：
    - 读取指定路径下的文件
    - 读取文件，处理数据
    - 将所有文件的数据合并成一个大的dataframe

In [22]:
path = './data/year'
file_list = os.listdir(path)
print(file_list)

['2004.csv', '2005.csv', '2006.csv', '2007.csv', '2008.csv', '2009.csv', '2010.csv', '2011.csv', '2012.csv', '2013.csv', '2014.csv', '2015.csv', '2016.csv', '疫情.csv']


In [23]:
df1 = pd.read_csv('./data/year/2004.csv',encoding='gbk',header=2)
df1.drop(['Unnamed: 0'],axis=1,inplace=True)
df1.head()

,地区,发病数,死亡数,发病率,死亡率
0,全国,49496.0,15.0,3.8077,0.0012
1,北京市,8.0,0.0,0.0540,NaN
2,天津市,13.0,0.0,0.1399,NaN
3,河北省,1923.0,0.0,2.8283,NaN
4,山西省,57.0,0.0,0.1720,NaN


In [24]:
file_year_name = os.path.splitext(file_list[0])[0]

# 添加列：年份
df1['年份'] = file_year_name
df1.head()

,地区,发病数,死亡数,发病率,死亡率,年份
0,全国,49496.0,15.0,3.8077,0.0012,2004
1,北京市,8.0,0.0,0.0540,NaN,2004
2,天津市,13.0,0.0,0.1399,NaN,2004
3,河北省,1923.0,0.0,2.8283,NaN,2004
4,山西省,57.0,0.0,0.1720,NaN,2004


In [ ]:
def merge_csv(path):
    data = pd.DataFrame()
    
    # 获取当前路径下的文件名清单
    file_list = os.listdir(path)
    
    for name_str in file_list:
        # 读取数据
        file_path = os.path.join(path,name_str)
        df = pd.read_csv(file_path,encoding='gbk',header=2)
        
        # 删除第一列
        df.drop(['Unnamed: 0'],axis=1,inplace=True)
        
        # 删除全是空值的行
        df.dropna(how='all',inplace=True)
        
        # 获取文件名的年份
        file_year_name = os.path.splitext(name_str)[0]
        
        # 添加列：年份
        df['年份'] = file_year_name
        
        # 合并多表数据
        data = data.append(df,ignore_index=True)
        
    return data

path = './data/year'
data = merge_csv(path)
data.to_csv('./data/year/疫情.csv')
data

## 4、批量修改文件名字

In [ ]:
# OS模块中的path

#  isdir(path)         判断指定路径是否存在且是一个目录
#  isfile(path)        判断指定路径是否存在且是一个文件
#  splitext(path)      分离文件名与扩展名，返回(f_name,f_extension)元祖
# join(path1[,path2...])  将path1与path2各部分组成一个路径名
#  exists(path)        判断指定路径（目录或文件）是否存在
#  getsize(file)       返回指定文件的尺寸，单位是字节
#  getctime(file)      返回指定文件的创建时间


In [ ]:
import os

# 返回绝对路径
result = os.path.abspath('./文件夹的名字')

# 判断是否是文件夹
result = os.path.isdir('./文件夹的名字')

result = os.path.isdir('./文件夹的名字.ipynb')

# 判断是否是文件
result = os.path.isfile('./文件夹的名字')

# 分离文件名与扩展名
result =os.path.splitext('批量修改文件名.ipynb')

# 判断指定路径（目录或文件）是否存在
result = os.path.exists('./文件夹的名字.ipynb')

# 返回指定文件的创建时间
result = os.path.getctime('./文件夹的名字')

print(result)

#拓展：将时间戳转化成制定格式的时间

import time
timearr = time.localtime(result)
mytime = time.strftime('%Y-%m-%d %H:%M:%S',timearr)
print(mytime)

In [ ]:
# os模块中的listdir

import os

# 获取到文件夹下的所有文件名字
result = os.listdir('./文件夹的名字')

oldpath = os.path.join('./文件夹的名字','SHEZHI.png')
newpath = os.path.join('./文件夹的名字','深圳.png') 
print(oldpath) 
                       
# 修改文件名的方法
os.rename(oldpath,newpath)


In [ ]:
# 1.传过来一个指定的路径
# 2.获取到这个路径下的文件名
# 3.修改文件名

## 5、批量生成word文件

In [16]:
from docx import Document
import pandas as pd

# 读取数据
df = pd.read_excel('./day5-word、Excel/姓名.xls','Sheet1')
df.columns.tolist()

['name', 'no', 'job', 'sale']

In [17]:
# #自定义替换的函数 
# #为什么需要两个参数:因为我们需要根据老数据,替换新数据
def repalce_text(old_text,new_text):
    
    paragraphs = document.paragraphs
    
    for paragraph in paragraphs:
        
        for run in paragraph.runs:
            #系统提供的内置方法 替换字符串的数据
            #最简单的方式
            run.text = run.text.replace(old_text,new_text)
 


for row in range(0,df.shape[0]):
    
    document= Document('./day5-word、Excel/模板.docx')
    #列索引
    my_col = df.columns.tolist()
    #遍历列
    for col in range(0,df.shape[1]):
        #取数据
        new_text = df.iloc[row,col]
        print(new_text)

        #取出要替换的老数据
        old_text = my_col[col]
        # 替换
        repalce_text(str(old_text),str(new_text))

    #保存数据   
    document.save('./day5-word、Excel/output/{}.docx'.format(df.iloc[row,0]))

张三
1001
财务主管
20000
李四
1002
数据分析师
30000
王五
1003
HR
15000


## 6、实例

In [5]:
import pandas as pd 
import numpy as np 

# dataframe.loc[使用的索引是我们自己定义的索引，即位置索引]
# dataframe.iloc[使用的索引是自带的数字索引]
# dataframe.loc[行索引，列索引]

### （1）处理一个文件

In [6]:
# 读取数据
df = pd.read_csv('./data/year/2004.csv',encoding='gbk',header=2)

# 删除第一列
df.drop(['Unnamed: 0'],axis=1,inplace=True)

# 删除全是空值的行
df.dropna(how='all',inplace=True)

# 添加列：年份
df.loc[:,'年份']=2004

df

,地区,发病数,死亡数,发病率,死亡率,年份
0,全国,49496.0,15.0,3.8077,0.0012,2004
1,北京市,8.0,0.0,0.0540,NaN,2004
2,天津市,13.0,0.0,0.1399,NaN,2004
3,河北省,1923.0,0.0,2.8283,NaN,2004
4,山西省,57.0,0.0,0.1720,NaN,2004
5,内蒙古,106.0,0.0,0.4446,NaN,2004
6,辽宁省,4.0,0.0,0.0096,NaN,2004
7,吉林省,4.0,0.0,0.0150,NaN,2004
8,黑龙江,17.0,0.0,0.0453,NaN,2004
9,上海市,15.0,0.0,0.1121,NaN,2004


### （2）封装一个函数，批量处理

思路：
    - 读取指定路径下的文件
    - 读取文件，处理数据
    - 将所有文件的数据合并成一个大的dataframe

In [3]:
import os  
path = './data/year'
file_list = os.listdir(path)
print(file_list)

['2004.csv', '2005.csv', '2006.csv', '2007.csv', '2008.csv', '2009.csv', '2010.csv', '2011.csv', '2012.csv', '2013.csv', '2014.csv', '2015.csv', '2016.csv', '疫情.csv']


In [9]:
import os  
def merge_csv(path):
    
    data = pd.DataFrame()
    
    # 获取当前路径下的文件名清单
    file_list = os.listdir(path)
    
    # 分别整理文件
    for name_str in file_list:
        
        # 读取数据
        file_path = os.path.join(path,name_str)
        df = pd.read_csv('./data/year/2004.csv',encoding='gbk',header=2)
        
        # 删除第一列
        df.drop(['Unnamed: 0'],axis=1,inplace=True)

        # 删除全是空值的行
        df.dropna(how='all',inplace=True)

        # 获取文件名的年份
        file_name = os.path.splitext(name_str)[0]
        
        # 添加列：年份
        df.loc[:,'年份']=file_name
        
        # 合并多表数据
        data = data.append(df,ignore_index = True)
    
    return data

path = './data/year'
data = merge_csv (path)  
data

,地区,发病数,死亡数,发病率,死亡率,年份
0,全国,49496.0,15.0,3.8077,0.0012,2004
1,北京市,8.0,0.0,0.0540,NaN,2004
2,天津市,13.0,0.0,0.1399,NaN,2004
3,河北省,1923.0,0.0,2.8283,NaN,2004
4,山西省,57.0,0.0,0.1720,NaN,2004
...,...,...,...,...,...,...
443,陕西省,1050.0,0.0,2.8703,NaN,疫情
444,甘肃省,978.0,0.0,3.7416,NaN,疫情
445,青海省,230.0,1.0,4.2627,0.0185,疫情
446,宁夏,193.0,0.0,3.3051,NaN,疫情


### （3）处理数据

In [10]:
# 去除地区列中的空格
data['地区']=[i.replace(" ","") for i in data['地区']]

# # 将黑龙江添加上省
data[data['地区']=="黑龙江"] = "黑龙江省"
data['地区'].value_counts()

江苏省     14
甘肃省     14
宁夏      14
广西      14
辽宁省     14
山西省     14
陕西省     14
北京市     14
内蒙古     14
安徽省     14
全国      14
湖南省     14
浙江省     14
广东省     14
新疆      14
湖北省     14
河南省     14
云南省     14
西藏      14
上海市     14
青海省     14
黑龙江省    14
吉林省     14
福建省     14
江西省     14
天津市     14
贵州省     14
河北省     14
重庆市     14
海南省     14
山东省     14
四川省     14
Name: 地区, dtype: int64

### （4）合并两表 
    -（data表和people表，基于【地区，年份 】 两列去合并）

In [12]:
# 读取people表
people = pd.read_csv('./data/people.csv',encoding='gbk')
people

,地区,2016年,2015年,2014年,2013年,2012年,2011年,2010年,2009年,2008年,2007年,2006年,2005年,2004年,2003年,2002年,2001年,2000年
0,北京市,2173,2171,2152,2115,2069,2019,1962,1860,1771,1676,1601,1538,1493,1456,1423,1385,1364
1,天津市,1562,1547,1517,1472,1413,1355,1299,1228,1176,1115,1075,1043,1024,1011,1007,1004,1001
2,河北省,7470,7425,7384,7333,7288,7241,7194,7034,6989,6943,6898,6851,6809,6769,6735,6699,6674
3,山西省,3682,3664,3648,3630,3611,3593,3574,3427,3411,3393,3375,3355,3335,3314,3294,3272,3247
4,内蒙古自治区,2520,2511,2505,2498,2490,2482,2472,2458,2444,2429,2415,2403,2393,2386,2384,2381,2372
5,辽宁省,4378,4382,4391,4390,4389,4383,4375,4341,4315,4298,4271,4221,4217,4210,4203,4194,4184
6,吉林省,2733,2753,2752,2751,2750,2749,2747,2740,2734,2730,2723,2716,2709,2704,2699,2691,2682
7,黑龙江省,3799,3812,3833,3835,3834,3834,3833,3826,3825,3824,3823,3820,3817,3815,3813,3811,3807
8,上海市,2420,2415,2426,2415,2380,2347,2303,2210,2141,2064,1964,1890,1835,1766,1713,1668,1609
9,江苏省,7999,7976,7960,7939,7920,7899,7869,7810,7762,7723,7656,7588,7523,7458,7406,7359,7327


In [13]:
# 整理people表

# 第一步，将地区中“自治区”去掉
people['地区'] = [i.replace('自治区','') for i in people['地区']]

# 第二步，反透视表操作 pd.melt()
year = list(people.columns).remove('地区')

result = pd.melt(people,
                id_vars=['地区'],   #不需要转换的列
                value_vars = year,  #需要转换的列
                var_name = ['年份'],
                value_name = '总人口数'
                )

result['年份'] = [i.replace('年','') for i in result['年份']]
result


# 数据透视操作
# pd.pivot_table(result,index='地区',columns='年份',values='总人口数')


# 第三步，两表合并
pd.merge(data,result,on=['年份','地区'])


,地区,发病数,死亡数,发病率,死亡率,年份,总人口数
0,北京市,8,0,0.054,NaN,2004,1493
1,天津市,13,0,0.1399,NaN,2004,1024
2,河北省,1923,0,2.8283,NaN,2004,6809
3,山西省,57,0,0.172,NaN,2004,3335
4,内蒙古,106,0,0.4446,NaN,2004,2393
...,...,...,...,...,...,...,...
385,陕西省,1050,0,2.8703,NaN,2016,3813
386,甘肃省,978,0,3.7416,NaN,2016,2610
387,青海省,230,1,4.2627,0.0185,2016,593
388,宁夏,193,0,3.3051,NaN,2016,675


In [14]:
import sys
sys.path.append('')
sys.path

['E:\\Jupyter-notebook\\数据分析基础',
 'D:\\ProgramData\\Anaconda3\\python37.zip',
 'D:\\ProgramData\\Anaconda3\\DLLs',
 'D:\\ProgramData\\Anaconda3\\lib',
 'D:\\ProgramData\\Anaconda3',
 '',
 'D:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'D:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'D:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'D:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'D:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\15622\\.ipython',
 '']